# 1. Preparación Notebook

## 1.1 Librerías

In [425]:
!pip install pulp

In [426]:
from google.colab import files
import json
import random
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus, LpBinary

## 1.2 Carga de datos

In [427]:
uploaded = files.upload()
filename = next(iter(uploaded))

Saving instance1.json to instance1 (11).json


In [428]:
with open(filename) as f:
    data = json.load(f)

# 2. Formulación del modelo

## 2.1 Conjuntos y parámetros

In [429]:
# Conjuntos
E = data['Employees']
D = data['Desks']
S = data['Days']
G = data['Groups']
Z = data['Zones']

# Parámetros
DesksZ_raw = data['Desks_Z']
DesksE = data['Desks_E']
EmployeesG = data['Employees_G']
DaysE_orig = data['Days_E']

In [430]:
# Mínimo 2 días presenciales por empleado; un 3er día solo si hay espacio
total_slots = len(D) * len(S)
min_required = 2 * len(E)
max_extra = total_slots - min_required

In [431]:
DesksZ = {}
for z, desks in DesksZ_raw.items():
    for d in desks:
        DesksZ[d] = z

## 2.2 Variables de decisión

In [432]:
model = LpProblem("Asignacion_Hibrida", LpMaximize)

# Variables
x = LpVariable.dicts("x", (E, D, S), cat=LpBinary)
z = LpVariable.dicts("z", (E, S), cat=LpBinary)
y = LpVariable.dicts("y", (G, S), cat=LpBinary)
w = LpVariable.dicts("w", (G, Z, S), cat=LpBinary)

In [433]:
u = LpVariable.dicts("u", (G, E, S), cat=LpBinary)  # z ∧ y
extra = LpVariable.dicts("extra", E, cat=LpBinary)

## 2.3 Restricciones

In [434]:
# R1: compatibilidad de escritorios (ya no limitamos días preferidos)
for e in E:
    for d in D:
        for s in S:
            if d not in DesksE[e]:
                model += x[e][d][s] == 0

# R3: un escritorio por día y empleado
for e in E:
    for s in S:
        model += lpSum(x[e][d][s] for d in D) <= 1

# R4: un escritorio no se comparte
for d in D:
    for s in S:
        model += lpSum(x[e][d][s] for e in E) <= 1

# R5: asignar 2 días por empleado, y un 3ro solo si extra[e] = 1
for e in E:
    model += lpSum(z[e][s] for s in S) >= 2
    model += lpSum(z[e][s] for s in S) <= 2 + extra[e]
model += lpSum(extra[e] for e in E) <= max_extra

# R6: un día de reunión por grupo
for g in G:
    model += lpSum(y[g][s] for s in S) == 1

# Eliminamos R7: ya no forzamos cobertura 100%

# R8: Solo permitir w[g][z][s] = 1 si hay reunión y alguien del grupo se sienta allí
for g in G:
    for s in S:
        for z_id in Z:
            empleados = EmployeesG[g]
            escritorios_en_zona = [d for d in D if DesksZ[d] == z_id]

            # Si alguien del grupo se sienta en esa zona el día de reunión, w = 1
            model += lpSum(x[e][d][s] for e in empleados for d in escritorios_en_zona) <= len(escritorios_en_zona) * w[g][z_id][s]

            # También, si w = 1, entonces al menos una persona se sentó ahí
            model += lpSum(x[e][d][s] for e in empleados for d in escritorios_en_zona) >= w[g][z_id][s]

# R9: ligar presencia y asignación de escritorio
for e in E:
    for s in S:
        model += lpSum(x[e][d][s] for d in D) == z[e][s]

# # R10: zonas sólo si hay reunión
# for g in G:
#     for z_id in Z:
#         for s in S:
#             model += w[g][z_id][s] <= y[g][s]

In [435]:
# R11: linealización de u[g,e,s] = z[e,s] ∧ y[g,s]
for g in G:
    for s in S:
        for e in EmployeesG[g]:
            model += u[g][e][s] <= z[e][s]
            model += u[g][e][s] <= y[g][s]
            model += u[g][e][s] >= z[e][s] + y[g][s] - 1

## 2.4 Función objetivo

In [436]:
alpha_orig = 2.0   # peso por día preferido
alpha_fill = 0.5   # peso por día no preferido
beta       = 1.2   # premio por reunión
gamma      = 0.5   # penalización por múltiples zonas
theta      = 1.0   # premio por cobertura de reunión

# Premio por asignar en días originales
orig_term = lpSum(
    alpha_orig * z[e][s]
    for e in E for s in DaysE_orig[e]
)

# Premio menor por días no preferidos
fill_term = lpSum(
    alpha_fill * z[e][s]
    for e in E for s in S if s not in DaysE_orig[e]
)

# Premio por reunión bien planificada y miembros asistiendo
meet_term   = beta  * lpSum(y[g][s] for g in G for s in S)
cover_term  = theta * lpSum(u[g][e][s] for g in G for e in EmployeesG[g] for s in S)

# Penalización por múltiples zonas en una reunión
zone_term = -gamma * lpSum(w[g][z_id][s] for g in G for z_id in Z for s in S)

# Premio por uso de asignaciones extra
bonus_term = 0.1 * lpSum(extra[e] for e in E)

# Objetivo completo
model += orig_term + fill_term + meet_term + cover_term + zone_term + bonus_term

## 2.5 Resolver modelo y mostrar resultados

In [437]:
# 7. Resolver y mostrar
model.solve()
print("Estado:", LpStatus[model.status], "\n")

Estado: Optimal 



In [438]:
print("\nAsignaciones completas:")
for e in E:
    print(f"\n🔹 Empleado {e}")
    assigned_days = 0
    for s in S:
        if z[e][s].varValue == 1:
            assigned_days += 1
            desk = next((d for d in D if x[e][d][s].varValue == 1), None)
            zone = DesksZ.get(desk, "N/A")
            print(f" ▸ Día {s} — Escritorio {desk} — Zona {zone}")
    print(f"  Total días asignados: {assigned_days}")


Asignaciones completas:

🔹 Empleado E0
 ▸ Día Mi — Escritorio D3 — Zona Z0
 ▸ Día V — Escritorio D6 — Zona Z1
  Total días asignados: 2

🔹 Empleado E1
 ▸ Día Ma — Escritorio D2 — Zona Z0
 ▸ Día Mi — Escritorio D2 — Zona Z0
  Total días asignados: 2

🔹 Empleado E2
 ▸ Día L — Escritorio D1 — Zona Z0
 ▸ Día Ma — Escritorio D3 — Zona Z0
 ▸ Día Mi — Escritorio D0 — Zona Z0
  Total días asignados: 3

🔹 Empleado E3
 ▸ Día L — Escritorio D2 — Zona Z0
 ▸ Día Mi — Escritorio D4 — Zona Z0
  Total días asignados: 2

🔹 Empleado E4
 ▸ Día L — Escritorio D0 — Zona Z0
 ▸ Día Ma — Escritorio D1 — Zona Z0
 ▸ Día Mi — Escritorio D1 — Zona Z0
  Total días asignados: 3

🔹 Empleado E5
 ▸ Día Ma — Escritorio D7 — Zona Z1
 ▸ Día J — Escritorio D0 — Zona Z0
  Total días asignados: 2

🔹 Empleado E6
 ▸ Día Mi — Escritorio D7 — Zona Z1
 ▸ Día J — Escritorio D4 — Zona Z0
  Total días asignados: 2

🔹 Empleado E7
 ▸ Día Ma — Escritorio D5 — Zona Z1
 ▸ Día Mi — Escritorio D5 — Zona Z1
 ▸ Día J — Escritorio D1 — Zona

In [439]:
print("\nZonas activadas y asistencia por grupo en su día de reunión:\n")

for g in G:
    for s in S:
        if y[g][s].varValue == 1:
            zonas_activas = [z_id for z_id in Z if w[g][z_id][s].varValue == 1]
            empleados_presentes = [e for e in EmployeesG[g] if z[e][s].varValue == 1]

            print(f"🔹 Grupo {g} — Día de reunión: {s}")
            print(f"  ▸ Zonas activadas: {', '.join(zonas_activas) if zonas_activas else 'Ninguna'}")
            print(f"  ▸ Empleados presentes: {', '.join(empleados_presentes) if empleados_presentes else 'Ninguno'}\n")


Zonas activadas y asistencia por grupo en su día de reunión:

🔹 Grupo G0 — Día de reunión: Mi
  ▸ Zonas activadas: Z0
  ▸ Empleados presentes: E0, E1, E2, E3, E4

🔹 Grupo G1 — Día de reunión: J
  ▸ Zonas activadas: Z0
  ▸ Empleados presentes: E5, E6, E7, E8, E9

🔹 Grupo G2 — Día de reunión: L
  ▸ Zonas activadas: Z0, Z1
  ▸ Empleados presentes: E10, E11, E13, E14

🔹 Grupo G3 — Día de reunión: V
  ▸ Zonas activadas: Z0
  ▸ Empleados presentes: E15, E16, E17, E18, E19



In [440]:
print("\nResumen diario de asignaciones:\n")

for s in S:
    empleados_dia = []
    escritorios_dia = []
    zonas_dia = []

    for e in E:
        for d in D:
            if x[e][d][s].varValue == 1:
                empleados_dia.append(e)
                escritorios_dia.append(d)
                zonas_dia.append(DesksZ[d])

    print(f"📅 Día {s}")
    print(f"  ▸ Total empleados asignados: {len(empleados_dia)}")

    if empleados_dia:
        for e, d, z in zip(empleados_dia, escritorios_dia, zonas_dia):
            print(f"    - Empleado {e} → Escritorio {d} (Zona {z})")
    else:
        print("    - No hay asignaciones.")

    print("-" * 40)


Resumen diario de asignaciones:

📅 Día L
  ▸ Total empleados asignados: 9
    - Empleado E2 → Escritorio D1 (Zona Z0)
    - Empleado E3 → Escritorio D2 (Zona Z0)
    - Empleado E4 → Escritorio D0 (Zona Z0)
    - Empleado E10 → Escritorio D7 (Zona Z1)
    - Empleado E11 → Escritorio D6 (Zona Z1)
    - Empleado E13 → Escritorio D4 (Zona Z0)
    - Empleado E14 → Escritorio D3 (Zona Z0)
    - Empleado E16 → Escritorio D8 (Zona Z1)
    - Empleado E17 → Escritorio D5 (Zona Z1)
----------------------------------------
📅 Día Ma
  ▸ Total empleados asignados: 9
    - Empleado E1 → Escritorio D2 (Zona Z0)
    - Empleado E2 → Escritorio D3 (Zona Z0)
    - Empleado E4 → Escritorio D1 (Zona Z0)
    - Empleado E5 → Escritorio D7 (Zona Z1)
    - Empleado E7 → Escritorio D5 (Zona Z1)
    - Empleado E9 → Escritorio D6 (Zona Z1)
    - Empleado E10 → Escritorio D4 (Zona Z0)
    - Empleado E14 → Escritorio D0 (Zona Z0)
    - Empleado E18 → Escritorio D8 (Zona Z1)
----------------------------------------
